<a href="https://colab.research.google.com/github/Viniciusp67/Consultor-de-Direito-Eleitoral/blob/main/cripto_gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance prophet tensorflow

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
from prophet import Prophet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping


In [4]:
# Baixar dados históricos do preço de fechamento do Ethereum até o final de 2023
end_date = '2024-06-30'
today = '2024-06-30'  # Data final para previsão
df_ethereum = yf.download('ETH-USD', '2016-01-01', end_date)
df_ethereum.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [5]:
# Preparar dados para Prophet e GRU
df = df_ethereum[["Date", "Adj Close"]]
df.rename(columns={'Date': 'ds', 'Adj Close': 'y'}, inplace=True)

<ipython-input-5-ae729887803a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Date': 'ds', 'Adj Close': 'y'}, inplace=True)


In [6]:
# Instanciar e ajustar o modelo Prophet
model_prophet = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True
)
model_prophet.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp740e5wl3/iazlhoac.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp740e5wl3/__vrtj_9.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=83613', 'data', 'file=/tmp/tmp740e5wl3/iazlhoac.json', 'init=/tmp/tmp740e5wl3/__vrtj_9.json', 'output', 'file=/tmp/tmp740e5wl3/prophet_modelgxzonkcr/prophet_model-20240701234610.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
23:46:10 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
23:46:12 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [7]:
# Criar um DataFrame futuro para prever os próximos 6 meses (até 30/06/2024)
future = model_prophet.make_future_dataframe(periods=(pd.to_datetime(today) - pd.to_datetime(end_date)).days)
forecast_prophet = model_prophet.predict(future)

In [8]:
# Normalizar dados
data = df['y'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

In [9]:
# Criar conjuntos de dados para o GRU
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 60
X, y = create_dataset(data_scaled, look_back)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))  # Adicionar dimensão de tempo para GRU

In [10]:
# Construir o modelo GRU
model_gru = Sequential()
model_gru.add(GRU(100, return_sequences=True, input_shape=(look_back, 1)))
model_gru.add(GRU(100, return_sequences=False))
model_gru.add(Dense(25))
model_gru.add(Dense(1))
model_gru.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Adicionar early stopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Treinar o modelo GRU com early stopping

num_epochs = 120  # Aumente o número de épocas conforme necessário
history = model_gru.fit(X, y, batch_size=1, epochs=num_epochs, validation_split=0.2, callbacks=[early_stopping])



Epoch 1/120
1891/1891 [==============================] - 116s 58ms/step - loss: 0.0013 - val_loss: 0.0010
Epoch 2/120
1891/1891 [==============================] - 106s 56ms/step - loss: 9.5679e-04 - val_loss: 0.0016
Epoch 3/120
1891/1891 [==============================] - 105s 56ms/step - loss: 7.3721e-04 - val_loss: 4.7949e-04
Epoch 4/120
1891/1891 [==============================] - 113s 60ms/step - loss: 8.9956e-04 - val_loss: 2.9174e-04
Epoch 5/120
1891/1891 [==============================] - 107s 57ms/step - loss: 5.7734e-04 - val_loss: 7.2904e-04
Epoch 6/120
1891/1891 [==============================] - 105s 56ms/step - loss: 6.1709e-04 - val_loss: 3.6613e-04
Epoch 7/120
1891/1891 [==============================] - 105s 55ms/step - loss: 5.3653e-04 - val_loss: 2.9477e-04
Epoch 8/120
1891/1891 [==============================] - 105s 55ms/step - loss: 5.1666e-04 - val_loss: 0.0014
Epoch 9/120
1891/1891 [==============================] - 104s 55ms/step - loss: 5.3229e-04 - val_loss: 2

In [ ]:
# Plotar o erro de treinamento e validação
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Erro de Treinamento')
plt.plot(history.history['val_loss'], label='Erro de Validação')
plt.xlabel('Épocas')
plt.ylabel('Erro')
plt.legend()
plt.title('Erro de Treinamento e Validação ao longo das Épocas')
plt.show()

In [ ]:
# Fazer previsões com GRU
def predict_future(model, data_scaled, look_back, n_future):
    predictions = []
    last_data = data_scaled[-look_back:]

    for _ in range(n_future):
        X_input = np.reshape(last_data, (1, look_back, 1))
        next_pred = model.predict(X_input)
        predictions.append(next_pred[0, 0])
        last_data = np.append(last_data[1:], next_pred, axis=0)

    return np.array(predictions)

In [ ]:
# Gerar previsões até 30/06/2024
n_future = (pd.to_datetime(today) - pd.to_datetime(end_date)).days
predictions_gru = predict_future(model_gru, data_scaled, look_back, n_future)
predictions_gru = scaler.inverse_transform(predictions_gru.reshape(-1, 1))

In [ ]:
# Convertendo end_date para um objeto datetime
end_date_dt = pd.to_datetime(end_date)

# Criar DataFrame para previsões do GRU
gru_forecast_index = pd.date_range(start=end_date_dt + pd.Timedelta(days=1), end=today, freq='D')
gru_forecast_df = pd.DataFrame({
    'ds': gru_forecast_index,
    'yhat_gru': predictions_gru.flatten()
})

In [ ]:
# Criar DataFrame combinado de previsões
combined_forecast = forecast_prophet[['ds', 'yhat']].copy()
combined_forecast = combined_forecast.merge(gru_forecast_df, on='ds', how='left')

In [ ]:
# Previsão combinada
weight_prophet = 0.5
weight_gru = 0.5
combined_forecast['yhat_combined'] = (weight_prophet * combined_forecast['yhat'] +
                                      weight_gru * combined_forecast['yhat_gru'])

In [ ]:
# Obter dados reais de 2024 para comparação
df_real_2024 = yf.download('ETH-USD', '2024-01-01', today)
df_real_2024.reset_index(inplace=True)
df_real_2024 = df_real_2024[["Date", "Adj Close"]]
df_real_2024.rename(columns={'Date': 'ds', 'Adj Close': 'y_real'}, inplace=True)

In [ ]:
# Merge os dados reais de 2024 com previsões combinadas
final_data = combined_forecast.merge(df_real_2024, on='ds', how='left')

In [ ]:
# Calcular erros
final_data['error'] = final_data['y_real'] - final_data['yhat_combined']
final_data['absolute_error'] = np.abs(final_data['error'])
final_data['squared_error'] = final_data['error'] ** 2

In [ ]:
# Número de linhas antes da remoção de NaNs
print(f"Número de linhas antes da remoção de NaNs: {len(final_data)}")

# Remover valores NaN
final_data_clean = final_data.dropna(subset=['error', 'absolute_error', 'squared_error'])

# Número de linhas após a remoção de NaNs
print("\nDataFrame após remoção de NaNs (final_data_clean):")
print(final_data_clean.head())
print(f"Número de linhas após remoção de NaNs: {len(final_data_clean)}")

In [ ]:
# Passo 2: Visualizar a dispersão do erro
plt.figure(figsize=(20, 10))

# Gráfico de dispersão dos erros
plt.subplot(2, 2, 1)
plt.scatter(final_data_clean['ds'], final_data_clean['error'], alpha=0.5)
plt.xlabel('Data')
plt.ylabel('Erro')
plt.title('Dispersão do Erro das Previsões Combinadas')

# Histograma do erro absoluto
plt.subplot(2, 2, 2)
plt.hist(final_data_clean['absolute_error'], bins=50, edgecolor='k')
plt.xlabel('Erro Absoluto')
plt.ylabel('Frequência')
plt.title('Distribuição do Erro Absoluto')

# Histograma do erro quadrático
plt.subplot(2, 2, 3)
plt.hist(final_data_clean['squared_error'], bins=50, edgecolor='k')
plt.xlabel('Erro Quadrático')
plt.ylabel('Frequência')
plt.title('Distribuição do Erro Quadrático')

# Gráfico de dispersão do erro em relação aos valores reais
plt.subplot(2, 2, 4)
plt.scatter(final_data_clean['y_real'], final_data_clean['error'], alpha=0.5)
plt.xlabel('Valor Real')
plt.ylabel('Erro')
plt.title('Erro vs Valor Real')

plt.tight_layout()
plt.show()

In [ ]:
# Visualizar dados históricos, previsões e dados reais
plt.figure(figsize=(20, 10))

# Plotar dados históricos
plt.plot(df['ds'], df['y'], label='Dados Históricos')

# Plotar previsões Prophet
plt.plot(combined_forecast['ds'], combined_forecast['yhat'], label='Previsão Prophet')

# Plotar previsões GRU
plt.plot(combined_forecast['ds'], combined_forecast['yhat_gru'], label='Previsão GRU')

# Plotar previsões combinadas
plt.plot(combined_forecast['ds'], combined_forecast['yhat_combined'], label='Previsão Combinada')

# Plotar dados reais de 2024
plt.plot(df_real_2024['ds'], df_real_2024['y_real'], label='Dados Reais 2024', linestyle='dotted')

plt.legend()
plt.xlabel('Data')
plt.ylabel('Preço (USD)')
plt.title('Previsões de Preço do Ethereum')
plt.grid(True)
plt.show()